
# 📦 고객 이탈 예측 분석 (Kaggle API 연동 최종본)

이 노트북은 `kaggle.json` API 키를 활용하여 Kaggle에서 데이터를 자동 다운로드하고,  
통신사 고객 이탈(Churn)을 예측하는 머신러닝 분석을 수행합니다.

---

## ✅ Usage Summary

1. [Kaggle 계정](https://www.kaggle.com/account)에서 `kaggle.json` API 키 다운로드  
2. 아래 코드 셀에서 `kaggle.json` 업로드  
3. 데이터 자동 다운로드 및 Unzip data  
4. 분석 코드 실행!


## 🔐 Step 1. Upload kaggle.json

In [ ]:

from google.colab import files
files.upload()  # kaggle.json 선택


## ⚙️ Step 2. Kaggle API Setup

In [ ]:

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


## 📥 Step 3. Download data 및 Unzip data

In [ ]:

!kaggle datasets download -d blastchar/telco-customer-churn
!unzip -o telco-customer-churn.zip -d telco_data


## 📄 Step 4. Load data

In [ ]:

import pandas as pd

df = pd.read_csv("telco_data/WA_Fn-UseC_-Telco-Customer-Churn.csv")
df.head()


## 📊 Step 5. Churn Ratio Visualization

In [ ]:

import matplotlib.pyplot as plt

df['Churn'].value_counts(normalize=True).plot(kind='bar', title='Churn Ratio')
plt.show()


## 🧹 Step 6. Data Preprocessing

In [ ]:

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df = df.dropna()

le = LabelEncoder()
for col in df.columns:
    if df[col].dtype == 'object' and col != 'customerID':
        df[col] = le.fit_transform(df[col])

X = df.drop(['customerID', 'Churn'], axis=1)
y = df['Churn']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


## 🤖 Step 7. Logistic Regression

In [ ]:

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

model_lr = LogisticRegression(max_iter=1000)
model_lr.fit(X_train, y_train)
y_pred_lr = model_lr.predict(X_test)

print("정확도:", accuracy_score(y_test, y_pred_lr))
print(classification_report(y_test, y_pred_lr))


## 🌲 Step 8. Random Forest

In [ ]:

from sklearn.ensemble import RandomForestClassifier

model_rf = RandomForestClassifier()
model_rf.fit(X_train, y_train)
y_pred_rf = model_rf.predict(X_test)

print("정확도:", accuracy_score(y_test, y_pred_rf))
print(classification_report(y_test, y_pred_rf))


## ⚡ Step 9. XGBoost 모델

In [ ]:

!pip install xgboost
from xgboost import XGBClassifier

model_xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
model_xgb.fit(X_train, y_train)
y_pred_xgb = model_xgb.predict(X_test)

print("정확도:", accuracy_score(y_test, y_pred_xgb))
print(classification_report(y_test, y_pred_xgb))


## 🔍 Step 10. 주요 Feature Importance

In [ ]:

import seaborn as sns

feat_importances = pd.Series(model_rf.feature_importances_, index=X.columns)
feat_importances.nlargest(10).plot(kind='barh')
plt.title("Top 10 Important Features")
plt.xlabel("Importance")
plt.show()



## ✅ Conclusion and Recommendations

- XGBoost 모델이 가장 높은 성능을 보였음
- 주요 이탈 원인은 `월 요금`, `계약 형태`, `고객 지원 품질`
- 실무 전략:
  - 장기 계약 유도
  - 고요금 고객 대상 맞춤 마케팅
  - 기술 지원 품질 개선
